## Table of Contents
* [Setup](#Setup)
	* [estimate cost](#estimate-cost)
	* [load dataset](#load-dataset)
* [Submitting HITs](#Submitting-HITs)
* [Retrieve results](#Retrieve-results)
* [Interact with workers](#Interact-with-workers)
* [Accepting and deleting HITs... careful with these](#Accepting-and-deleting-HITs...-careful-with-these)


In [19]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import re
import pickle
import boto
import json
import os
import random
from copy import deepcopy
from tqdm import tqdm

from IPython.core.display import HTML

from keysTkingdom import mturk_ai2

from mturk_utils.mturk import MTurk
from mturk_utils.mturk import pickle_this, unpickle_this

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from mturk_utils.annotation_collection import prepare_hit
from mturk_utils.annotation_collection import write_task_page
from mturk_utils.mturk import expected_cost

# Submitting HITs

In [37]:
turk_account = mturk_ai2
rw_host='mechanicalturk.amazonaws.com'
# amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key, host=rw_host)
amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key)
amt_con.get_account_balance()

$10,000.00

In [41]:
static_params = {
    'title': "task title",
    'description': "task description",
    'keywords': ['task keyword'],
    'frame_height': 1000,
    'amount': 0.01,
    'duration': 3600 * 1,
    'lifetime': 3600 * 24 * 2,
    'max_assignments': 3,
    'locales': ['US', 'CA', 'AU', 'NZ', 'GB']
}

In [42]:
working_corrected = unpickle_this('/Users/schwenk/wrk/animation_gan/turk_annotation_task/stage_3/pickled_data/corrected_descriptions.pkl')

In [43]:
sent_len_threshold = 30
current_batch_short_sent = {vid: desc for vid, desc in working_corrected.items() if sent_len_threshold > len(desc.split())}
current_batch_very_long_sent = {vid: desc for vid, desc in working_corrected.items() if sent_len_threshold <= len(desc.split())}

In [44]:
build_hit_group = [prepare_hit(vid, desc, static_params) for vid, desc in tqdm(list(current_batch_short_sent.items())[:5])]

100%|██████████| 5/5 [00:00<00:00, 177.92it/s]


In [45]:
write_task_page(random.choice(build_hit_group)['html'])

In [46]:
expected_cost(build_hit_group, static_params, amt_con)

0.15

creates HITs: 

In [47]:
hit_group = [amt_con.create_html_hit(single_hit) for single_hit in tqdm(build_hit_group)]

100%|██████████| 5/5 [00:01<00:00,  4.26it/s]


# Retrieve results

In [9]:
# %%time
# all_hits = amt_con.get_all_hits()

In [18]:
# %%time
# all_hits = amt_con.get_reviewable_hits(detailed=False)

In [115]:
start_date = (2017, 10, 16)
end_date = (2017, 10, 25)
recent_hits = filter_hits_by_date(all_hits, start_date, end_date)

In [81]:
%%time
new_4b_assignemnts_10_26 = get_assignments(amt_con.connection, new_4b_hits)

CPU times: user 48.7 s, sys: 2.55 s, total: 51.3 s
Wall time: 35min 28s


In [83]:
pickle_this(new_4b_assignemnts_10_26, 'new_4b_assignemnts_10_26.pkl')

# Interact with workers

## reject assignments and ban workers

In [37]:
def ban_bad_workers(mturk_connection, worker_ids):
    for worker in worker_ids:
        reason_for_block = """
        Your HITs contained many frames with characters, but they were marked 'empty frame'
        """
        print('blocking ' + str(worker))
        mturk_connection.block_worker(worker, reason_for_block)

In [38]:
workers_to_ban = unpickle_this('workers_to_ban_prod_1_4.pkl')

In [40]:
ban_bad_workers(amt_con.connection, list(workers_to_ban))

blocking A2PU4YNWITAQVL
blocking A273GB71FE8RIV
blocking A3R6KO03P9QLNO
blocking A2X7BE2FSPUKCK
blocking A1QP6651C4Q8NS
blocking A2CXEAMWU2SFV3
blocking A2NJTYYY37DNO
blocking A1CF1W8CP0DHB0
blocking A3UPV16IECELHV
blocking A2L97S8Z5FZK5N
blocking A1IOMFFEKCWOIT
blocking A15DOG6AXQ7LGY
blocking A9ALF1SODWQE5
blocking A1RULXRXR2XF6M
blocking A13ISH3PAGHO0A


In [46]:
to_reject = unpickle_this('assignments_to_reject_prod_1_4.pkl')

In [43]:
def reject_assignments(mturk_connection, assignments_to_reject):
    feedback_message = """
    Your HITs contained many frames with characters, but they were marked 'empty frame'
    """
    reject_count = len(assignments_to_reject)
    for assignment_id in tqdm(assignments_to_reject):
        try:
            mturk_connection.reject_assignment(assignment_id, feedback_message)
        except boto.mturk.connection.MTurkRequestError:
            print('assignment ' + str(assignment_id) + ' already accepted or rejected')

    return reject_count

In [45]:
# reject_assignments(amt_con.connection, list(to_reject))

In [458]:
def pay_bonuses(bonuses_to_pay):
    total_payed = 0
    bonus_reason_template = 'For Flinstones character labels written.'
    for aid, vals in tqdm(bonuses_to_pay.items()):
        n_chars = vals['count']
        wid = vals['worker_id']
        bonus_ammount = boto.mturk.price.Price(0.01 * n_chars)
        total_payed += bonus_ammount.amount
        bonus_reason = bonus_reason_template
        amt_con.connection.grant_bonus(wid, aid, bonus_ammount, bonus_reason)
    return total_payed

In [455]:
# bonuses = unpickle_this('bonuses_to_pay_3.pkl')

In [460]:
# pay_bonuses(bonuses)

# Accepting and deleting HITs

Uncomment only when ready to accept or delete hits

reject assignments carefully

In [8]:
# accept_results = unpickle_this('pickeled_results/prod_batch_1_4.pkl')

In [545]:
e_count = 0
for assignment in tqdm(accept_results):
    try:
        amt_con.approve_assignment(assignment)
    except boto.mturk.connection.MTurkRequestError as e:
        e_count += 1

100%|██████████| 31407/31407 [1:28:56<00:00,  5.67it/s]  


disable hits

In [178]:
# _ = [amt_con.disable_hit(hit) for hit in tqdm(all_hits)]

deletes hits

In [20]:
# amt_con.delete_all_hits()